In [4]:
from tadf_models.models import *
from chemdataextractor.doc import Document, Citation
import json
import importlib.resources
from chemdataextractor.model import ThemeCompound
import os
from pprint import pprint

f:\test-envs\cde_tadf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tadf_blacklist = importlib.resources.read_text('tadf_models', 'tadf_blacklist_6_more_abbrev_enriched', encoding='utf8')
element_blacklist = importlib.resources.read_text('tadf_models', 'elements', encoding='utf8')
tadf_blacklist = tadf_blacklist.split('\n')
element_blacklist = element_blacklist.split('\n')
ThemeCompound.name_blacklist = ThemeCompound.name_blacklist + tadf_blacklist + element_blacklist

In [6]:
class TADFExtractor():
    """Class that extracts a document and saves records in JSON format."""

    def __init__(self, paper_root, models):
        """
        Initalise the class with the directory and name of the document, the models to extract, and the save directory.
        :param str paper_root: directory that stores the paper.
        :param str save_root: directory that the JSON results are written into.
        :param str filename: The name of output file name.
        :param list[chemdataextractor.models.BaseModel] models: list of ChemDataExtractor models to extract.
        """
        self.dic = None
        self.paper_root = paper_root
        self.models = models
        self.count = 0

    def is_incomplete_paper(self, fstring):
        """
        Method that check if a document is complete.
        :param str fstring: document in plain text string.
        :returns: if the document is complete or not.
        :rtype: Boolean.
        """
        print('Checking completeness.')
        if "(Note: The full text of this document is currently only available in the" in fstring:
            # RSC case
            return True
        elif "<xocs:rawtext" in fstring:
            # Elsevier case
            return True
        return False

    def load_document(self, file):
        """
        Method that load a document into a chemdataextractor.doc.Document object.
        :param str file: path to the document.
        :returns: the Document object and if the document is incomplete.
        :rtype: chemdataextractor.doc.Document, Boolean
        """

        d = Document.from_file(file)
        incomplete = False
        with open(file, encoding="utf8") as f:
            fstring = f.read()
        if self.is_incomplete_paper(fstring):
            incomplete = True

        return d, incomplete

    def extraction(self, file):
        """
        Method that load a extract records from a document and save them in JSON.
        :param str file: path to the document.
        """

        print(f'Attempting to extract {file}.')

        if True:
            doc, incomplete = self.load_document(os.path.join(self.paper_root, file))

            if incomplete:
                print(f'{file} is incomplete!')
                return
            
            else:
                print(f"{file} is complete, extracting.")

                try:
                    doi = doc.metadata.doi

                except IndexError:
                    print("MetaData is Empty.")
                    doi = file.name

                doc.models = self.models
                doc.skip_elements = [Citation]
                rough = doc.records

                records = []
                for r in rough:
                    self.dic = r.serialize()
                    self.dic['doi'] = doi
                    self.dic['record_method'] = r.record_method
                    records.append(self.dic)
                    self.count += 1

                print(f"{self.count} records in total")
                print(f"{file} extracted.")
                return records

In [7]:
mdb = TADFExtractor(paper_root="tadf_models", 
                    models=[PhotoluminescenceQuantumYield, 
                            STSplit,
                            EmissionWavelength,
                            DelayedLifetime])
records = mdb.extraction("test.html")
print()
pprint(records)

Attempting to extract test.html.
Checking completeness.
test.html is complete, extracting.
Initialising AllenNLP model ✔   
12 records in total
test.html extracted.

[{'STSplit': {'compound': {'ThemeCompound': {'names': ['2,3-Di(9H-carbazol-9-yl)thianthrene '
                                                       '5,5,10,10-tetraoxide',
                                                       '2,3-di(9H-carbazol-9-yl)thianthrene '
                                                       '5,5,10,10-tetraoxide',
                                                       'DCZ-TTR'],
                                             'roles': ['nesting theme']}},
              'raw_units': 'eV',
              'raw_value': '0.03',
              'specifier': 'singlet',
              'units': 'ElectronVolt^(1.0)',
              'value': [0.03]},
  'doi': '10.1039/C7TC00681K',
  'record_method': 'QuantityModelTemplateParser'},
 {'STSplit': {'compound': {'ThemeCompound': {'names': ['2-(9H-Carbazol-9-yl)thian